## 환경 설정

In [ ]:
# 주피터
import os

# os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

In [ ]:
# 코랩
from google.colab import drive
drive.mount("/content/drive")

import os
path = '/content/drive/My Drive/projects/NLP-StockMarket/model_fin/' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

In [ ]:
# !pip install -q transformers datasets

In [126]:
import pandas as pd
from datasets import load_dataset, ReadInstruction
from datetime import datetime, timedelta
import numpy as np
from transformers import AutoTokenizer, TFBertForSequenceClassification, BertTokenizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# from .autonotebook import tqdm as notebook_tqdm


from transformers import AutoTokenizer, AutoModelForMaskedLM

## 데이터 불러오기

### 종목명 선택 및 뉴스,토론방, 유튜브 데이터 통합

In [5]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [45]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col = 0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col = 0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col = 0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col = 0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col = 0)

In [46]:
# 데이터 통합
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])

# 'Date' 타입이 int 이므로 datetime으로 변환
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))

# 합쳐진 데이터들의 인덱스 재설정
news_df.sort_values('Date', ignore_index=True, inplace=True)

In [47]:
# 2021년 주식 데이터가 1월 4일부터 있어서 슬라이싱
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0] : ]
news_df.head(2)

,Date,Title
639,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결
640,2021-01-04,환율 하락 전환 1086 2 감소0 1원


### 주가 데이터

In [48]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head(2)

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24


In [49]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


In [50]:
stock_df['updown'] = 0

## 데이터 프레임 합치기

In [51]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

In [54]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률', 'updown']
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
print(len(df))
df.dropna(inplace =True)
df.head()

429241


,Date,Title,주가의 날짜,등락률,updown
0,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,0
1,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,0
2,2021-01-04,코스피 1 03p 0 04 오른 2874 50 출발 원 달러 환율 1 2원 ...,2021-01-05,2.24,0
3,2021-01-04,韓증시 사상최고치 시작 개장식선 안정적 시장운영에 방점,2021-01-05,2.24,0
4,2021-01-04,SK바이오팜 아벨 지분 매각으로 5천500만 달러 자본이득,2021-01-05,2.24,0


In [55]:
df = df.iloc[: 4500, :]             ## 임시!!!
len(df)

4500

## 감성사전 load

In [56]:
sentiment_csv = pd.read_csv('./sentiment dictionary.csv', index_col = 0)
sentiment_csv.head()

,pos,mid,neg
0,방긋,아직,회의적
1,상회,보통,바닥
2,신선,vs,이탈
3,신박,중립,떨어지
4,투혼,관망,안좋게


In [57]:
pos_li = sentiment_csv['pos'].dropna().values
mid_li = sentiment_csv['mid'].dropna().values
neg_li = sentiment_csv['neg'].dropna().values

### 감성 지수 계산하는 sentimental_score() 

In [58]:
### binary_sentimental_score
def binary_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['감성지수'] = 0
    
    # SP 등락률에 따른 updown 계산
    df_result.loc[df_result.query('등락률 >= 0').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 < 0').index, 'updown'] = -1

    # 감성 지수는 긍정 : 1, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999    
    
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1').index, '감성지수'] = -1
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1').index].index) : 
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]
        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 0 :
            df_result.loc[i,'감성지수'] = 1
        else :
            df_result.loc[i,'감성지수'] = -1
    df_result = pd.concat([df_result.loc[df_result.query('감성지수 != 999').index]], ignore_index = True)
    return df_result

In [59]:
### multi_sentimental_score
def multi_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['Mid'] = 0
    df_result['감성지수'] = 0
    
    # SP 등락률에 따른 updown 계산
    df_result.loc[df_result.query('등락률 > 1').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 < -1').index, 'updown'] = -1
    
    # 감성 지수는 긍정 : 1, 중립 : 0, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999    
    
################################################################################################
    # 0  : 없음, 1: 있음
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    print('중립 단어 검색중')
    for mid in tqdm(mid_li) :
        str_expr = f"Title.str.contains('{mid}')"
        df_result.loc[df_result.query(str_expr).index, 'Mid'] = 1
    
    
################################################################################################
    
    # 모든 종류의 단어가 검출 되면 제외
    df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 1').index, '감성지수'] = 999
    
    # 중립 단어가 검출되면 중립
    df_result.loc[df_result.query('Mid == 1').index, '감성지수'] = 0
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0 and Mid == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1 and Mid == 0').index, '감성지수'] = -1
    
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 0').index].index) : 
        
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]
        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 1 :
            df_result.loc[i,'감성지수'] = 1
        elif updown < -1 :
            df_result.loc[i,'감성지수'] = -1
        else :
            df_result.loc[i,'감성지수'] = 0
    return df_result

In [60]:
df_result = multi_sentimental_score(df)

긍정 단어 검색중


100%|███████████████████████████████████████████████████████████████████████████████| 648/648 [00:02<00:00, 278.81it/s]


부정 단어 검색중


100%|█████████████████████████████████████████████████████████████████████████████| 1001/1001 [00:03<00:00, 281.36it/s]


중립 단어 검색중


100%|█████████████████████████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 259.71it/s]


긍정 부정 둘 다 있는 경우 처리중


100%|██████████████████████████████████████████████████████████████████████████████| 782/782 [00:00<00:00, 1414.90it/s]


## 데이터 분리

In [84]:
train_df, test_df = train_test_split(df_result, shuffle = False, test_size = 0.1)

In [85]:
print(train_df.shape)
print(test_df.shape)

(4050, 9)
(450, 9)


In [86]:
train_df.to_csv("./data/train.csv",encoding='utf-8-sig',index = False)
test_df.to_csv("./data/test.csv",encoding='utf-8-sig',index = False)

In [87]:
train_data = './data/train.csv'
test_data = './data/test.csv'

In [88]:
from datasets import load_dataset, ReadInstruction

dataset = load_dataset('csv', data_files={'train': train_data, 'test': test_data})

Using custom data configuration default-7f5f2aeeea528611


Extracting data files: 100%|████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 500.22it/s]


Dataset csv downloaded and prepared to C:\Users\jinyo\.cache\huggingface\datasets\csv\default-7f5f2aeeea528611\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 95.87it/s]


In [89]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Date', 'Title', '주가의 날짜', '등락률', 'updown', 'Pos', 'Neg', 'Mid', '감성지수'],
        num_rows: 4050
    })
    test: Dataset({
        features: ['Date', 'Title', '주가의 날짜', '등락률', 'updown', 'Pos', 'Neg', 'Mid', '감성지수'],
        num_rows: 450
    })
})

In [90]:
# labels = [label for label in dataset['train'].features.keys() if label in ['Pos','Neg','Mid']]
# id2label = {idx:label for idx, label in enumerate(labels)}
# label2id = {label:idx for idx, label in enumerate(labels)}

# print(labels)
# print(id2label)
# print(label2id)

In [91]:
# from transformers import AutoTokenizer
# import numpy as np

# tokenizer = AutoTokenizer.from_pretrained("klue/bert-base") # KlueBERT의 tokenizer를 활용합니다.

# def preprocess_data(examples):
#     # 배치화된 텍스트를 받습니다.
#     text = examples["Title"]
#     # 인코딩 합니다.
#     encoding = tokenizer(text, padding="max_length", truncation=True, max_length=100)
#     # 라벨을 배치로 만들어줍니다.
#     labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
#     # numpy array로 만들기 위해 0 매트릭스를 만들어줍니다.
#     labels_matrix = np.zeros((len(text), len(labels)))
#     # 채웁니다.
#     for idx, label in enumerate(labels):
#         labels_matrix[:, idx] = labels_batch[label]
#         encoding["labels"] = labels_matrix.tolist()
#         return encoding

In [ ]:
# encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

In [ ]:
# encoded_dataset.set_format("torch")

## 토크나이징

In [79]:
max_seq_len = 128

In [127]:
from transformers import AutoTokenizer
import numpy as np

# tokenizer = AutoTokenizer.from_pretrained("klue/bert-base") # KlueBERT의 tokenizer를 활용합니다.
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert") # FinBERT

Downloading: 100%|████████████████████████████████████████████████████████████████████| 336/336 [00:00<00:00, 56.8kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 140k/140k [00:00<00:00, 245kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████| 287k/287k [00:00<00:00, 373kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 33.9kB/s]


In [129]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [130]:
train_X, train_y = convert_examples_to_features(dataset['train']['Title'], dataset['train']['감성지수'], max_seq_len=128, tokenizer=tokenizer)

  0%|                                                                                         | 0/4050 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\jinyo\.conda\envs\py123\lib\site-packages\transformers\tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 5

In [131]:
test_X, test_y = convert_examples_to_features(dataset['test']['Title'],  dataset['test']['감성지수'], max_seq_len=128, tokenizer=tokenizer)

  0%|                                                                                          | 0/450 [00:00<?, ?it/s]C:\Users\jinyo\.conda\envs\py123\lib\site-packages\transformers\tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 450/450 [00:00<00:00, 7428.81it/s]


In [132]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [    2  3099  5719  8473 10120 10016  5070 12491  3389  6602  6066  5128
  4119  5436 10330 11979     3     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

## 모델-KlueBERT

### multi

In [133]:
# model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=3, from_pt=True)
model = TFBertForSequenceClassification.from_pretrained("snunlp/KR-FinBert", num_labels=3, from_pt=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.categorical_hinge
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 655/655 [00:00<00:00, 164kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 387M/387M [00:19<00:00, 20.9MB/s]
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and a

In [124]:
# early_stopping = EarlyStopping(
#     monitor="val_accuracy", 
#     min_delta=0.001,
#     patience=2)

In [134]:
model.fit(
    train_X, train_y, epochs=2, batch_size=32, validation_split=0.2
#     ,callbacks = [early_stopping]
)

Epoch 1/2
  1/102 [..............................] - ETA: 46:57 - loss: 29.3469 - accuracy: 0.1875

KeyboardInterrupt: 

## 성능평가

In [ ]:
model.evaluate()

snunlp/KR-FinBert-SC

In [ ]:
model.save_model("./model/checkpoint-best-model")

In [ ]:
text = "삼성SDI 적자 전환"

encoding = tokenizer(text, return_tensors="pt")

outputs = trainer.model(**encoding)   # (**encoding) input으로 encoding된 숫자값이 들어가도록 함

In [ ]:
logits = outputs.logits # labels에 대해 리뷰를 기준으로 모델이 뱉은 각각의 숫자값 (실수)-> logits이라고 함
logits.shape

In [ ]:
# apply sigmoid + threshold   # 확률값 중에 뭐를 기준으로 해서 결과를 뱉어 낼거냐. threshold가 높을수록 아주 높은 값만을 라벨링하게 됨
sigmoid = torch.nn.Sigmoid()   #  sigmoid 함수 호출
probs = sigmoid(logits.squeeze().cpu())  # 각각에 대해 확률값으로 바꿔주는 것 
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.1)] = 1  # threshold 값 : 0.1 --> probs이 0.1 이상인 값들만 1로 바꾸고, 나머지는 0으로 남김  -->  1로 해당하는 값이 최종 라벨링된 값
                                                                            # threshold 값 정하는 기준 : 경험적 or ROC 커브(threshold를 계속 바꿔가면서 그래프에 점을 찍어서 curve를 그린것, FPR가 낮고-TPR가 높은 것에 가까운 점을 갖는(AUC를 가장 높게 만드는) threshold를 찾는것)

predicted_labels = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
print(predicted_labels)

In [ ]:
KR-FinBert

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert")
model = AutoModelForMaskedLM.from_pretrained("snunlp/KR-FinBert")